# Valence value regression based on Deap Dataset

## 0. This notebook is based on DEAP database

Anyone should refer to DEAP team first

@article{koelstra2012deap,
  title={Deap: A database for emotion analysis; using physiological signals},
  author={Koelstra, Sander and Muhl, Christian and Soleymani, Mohammad and Lee, Jong-Seok and Yazdani, Ashkan and Ebrahimi, Touradj and Pun, Thierry and Nijholt, Anton and Patras, Ioannis},
  journal={IEEE Transactions on Affective Computing},
  volume={3},
  number={1},
  pages={18--31},
  year={2012},
  publisher={IEEE}
}

## 1. Dependency
* numpy
* pyEEG
* sciki-learn

In [1]:
!pip install numpy
!pip install pandas

!pip install scikit-learn


!pip install tensorflow

  Using cached numpy-2.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached scikit_learn-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.4 MB)
Using cached joblib-1.4.2-py3-no

In [2]:
!pip install git+https://github.com/forrestbao/pyeeg.git


  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-msu0jfns
  Running command git clone --filter=blob:none --quiet https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-msu0jfns
  Resolved https://github.com/forrestbao/pyeeg.git to commit a6c18bb093e4748f9d9c208535a6ae024a0802b8
  Preparing metadata (setup.py) ... done


In [3]:
!pip install tensorflow

In [2]:
import numpy as np
#import pyeeg as pe
import pickle as pickle
import pandas as pd
import math

from sklearn import svm
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import os
import tensorflow as tf
import time

2024-08-16 12:37:02.889870: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-16 12:37:02.892720: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-16 12:37:02.896681: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-16 12:37:02.910178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 12:37:02.931357: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

## 2. Global Variables setup
File Name data\SXX.dat, XX \in [0,31]
* data: 40 x 40 x 8064: trial x channel x data
* label: 40 x 4: video/trial x label (valence, arousal, dominance, liking)

Channel Indice: {
* 1 : AF3; 2: F3; 3: F7; 4: FC5; 7: T7; 11: P7; 13: O1
* 17: AF4; 19: F4; 20: F8; 21: FC6; 25: T8; 29: P8; 31: O2 }

In [3]:
eeg_channels = np.array(["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8", "PO4", "O2"])
channel_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
peripheral_channels = np.array(["hEOG", "vEOG", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])
band = [4,8,12,30,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
#List of subjects

## 3. FFT with pyeeg
* [4-8]: theta band
* [8-12]: alpha band
* [12-30]: beta band 
* [30-45]: low gamma band
* 

In [4]:
from sklearn.decomposition import FastICA
def raw_ica (sub, channel_indices, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = list([])
    with open('../data/s' + sub + '.dat', 'rb') as file:
        
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            data = data[:32]
            data = np.transpose(data)
            ica = FastICA(n_components=32,tol=1,max_iter=300)
            data_ica = ica.fit_transform(data)
            data_ica = np.transpose(data_ica)
            start = 0;

            while start + window_size < data_ica.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel_indices:
                    X = data_ica[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    
                    meta_data.append(X)  # Append the power values

                meta_array = list([np.array(meta_data), np.array(labels)])  # Create a homogeneous list
                meta.append(meta_array) 

                #meta_array.append(np.array(meta_data))
                #meta_array.append(labels)

                #meta.append(np.array(meta_array))

                start = start + step_size
                
    meta = np.array(meta,dtype=object)
    np.save('raw_ica\s' + sub, meta, allow_pickle=True, fix_imports=True)
def testing (M, L, model):
    '''
    arguments:  M: testing dataset
                L: testing dataset label
                model: scikit-learn model

    return:     void
    '''
    output = model.predict(M[0:78080:32])
    label = L[0:78080:32]

    k = 0
    l = 0

    for i in range(len(label)):
        k = k + (output[i] - label[i])*(output[i] - label[i]) #square difference 

        #a good guess
        if (output[i] > 5 and label[i] > 5):
            l = l + 1
        elif (output[i] < 5 and label[i] <5):
            l = l + 1

    print ("l2 error:", k/len(label), "classification accuracy:", l / len(label),l, len(label))

In [11]:

def load_raw (sub, channel_indices, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = list([])
    with open('../data/s' + sub + '.dat', 'rb') as file:
        
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            data = data[:32]
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel_indices:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    
                    meta_data.append(X)  # Append the power values

                meta_array = list([np.array(meta_data), np.array(labels)])  # Create a homogeneous list
                meta.append(meta_array) 

                #meta_array.append(np.array(meta_data))
                #meta_array.append(labels)

                #meta.append(np.array(meta_array))

                start = start + step_size
                
    meta = np.array(meta,dtype=object)
    np.save('raw\s' + sub, meta, allow_pickle=True, fix_imports=True)

def testing (M, L, model):
    '''
    arguments:  M: testing dataset
                L: testing dataset label
                model: scikit-learn model

    return:     void
    '''
    output = model.predict(M[0:78080:32])
    label = L[0:78080:32]

    k = 0
    l = 0

    for i in range(len(label)):
        k = k + (output[i] - label[i])*(output[i] - label[i]) #square difference 

        #a good guess
        if (output[i] > 5 and label[i] > 5):
            l = l + 1
        elif (output[i] < 5 and label[i] <5):
            l = l + 1

    print ("l2 error:", k/len(label), "classification accuracy:", l / len(label),l, len(label))

In [12]:
window_size = 256
step_size = 128
for subjects in subjectList:
    load_raw (subjects, channel_indices, band, window_size, step_size, sample_rate)


In [5]:
window_size = 256
step_size = 128
for subjects in subjectList:
    raw_ica (subjects, channel_indices, band, window_size, step_size, sample_rate)


In [7]:
window_size = 640
step_size = 40
for subjects in subjectList:
    FFT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)
window_size = 1280
step_size = 80
for subjects in subjectList:
    FFT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)

KeyboardInterrupt: 

## 3.Segment of preprocessed data
* training dataset: 80 %
 *testing dataset: 20%

Agrithom pool:
* Support Vector Machine (which kernal?)
* knn
*RF

Best practice could be refered to this paper: 

@article{alarcao2017emotions,
  title={Emotions recognition using EEG signals: A survey},
  author={Alarcao, Soraia M and Fonseca, Manuel J},
  journal={IEEE Transactions on Affective Computing},
  year={2017},
  publisher={IEEE}
}

## 4.Regression
### 0. Loading Training and Testing dataset


### 1. Support Vector Regression
* default setting, l1 error: 1.621761042477756 classification error: 0.6057377049180328 1478 2440